# Tracing for Different Types of Runs

### Types of Runs

LangSmith supports many different types of Runs - you can specify what type your Run is in the @traceable decorator. The types of runs are:

- LLM: Invokes an LLM
- Retriever: Retrieves documents from databases or other sources
- Tool: Executes actions with function calls
- Chain: Default type; combines multiple Runs into a larger process
- Prompt: Hydrates a prompt to be used with an LLM
- Parser: Extracts structured data

### Setup

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv()

True

### LLM Runs for Chat Models

LangSmith provides special rendering and processing for LLM traces. In order to make the most of this feature, you must log your LLM traces in a specific format.

For chat-style models, inputs must be a list of messages in OpenAI-compatible format, represented as Python dictionaries or TypeScript object. Each message must contain the key role and content.

The output is accepted in any of the following formats:

- A dictionary/object that contains the key choices with a value that is a list of dictionaries/objects. Each dictionary/object must contain the key message, which maps to a message object with the keys role and content.
- A dictionary/object that contains the key message with a value that is a message object with the keys role and content.
- A tuple/array of two elements, where the first element is the role and the second element is the content.
- A dictionary/object that contains the key role and content.
The input to your function should be named messages.

You can also provide the following metadata fields to help LangSmith identify the model and calculate costs. If using LangChain or OpenAI wrapper, these fields will be automatically populated correctly.
- ls_provider: The provider of the model, eg "openai", "anthropic", etc.
- ls_model_name: The name of the model, eg "gpt-4o-mini", "claude-3-opus-20240307", etc.

In [2]:
from langsmith import traceable

inputs = [
  {"role": "system", "content": "You are a helpful assistant."},
  {"role": "user", "content": "I'd like to book a table for two."},
]

output = {
  "choices": [
      {
          "message": {
              "role": "assistant",
              "content": "Sure, what time would you like to book the table for?"
          }
      }
  ]
}

# Can also use one of:
# output = {
#     "message": {
#         "role": "assistant",
#         "content": "Sure, what time would you like to book the table for?"
#     }
# }
#
# output = {
#     "role": "assistant",
#     "content": "Sure, what time would you like to book the table for?"
# }
#
# output = ["assistant", "Sure, what time would you like to book the table for?"]

@traceable(
  # TODO: Add an run_type="llm", and metadata for ls_provider, and ls_model_name
)
def chat_model(messages: list):
  return output

chat_model(inputs)

{'choices': [{'message': {'role': 'assistant',
    'content': 'Sure, what time would you like to book the table for?'}}]}

### Handling Streaming LLM Runs

For streaming, you can "reduce" the outputs into the same format as the non-streaming version. This is currently only supported in Python.

In [3]:
def _reduce_chunks(chunks: list):
    all_text = "".join([chunk["choices"][0]["message"]["content"] for chunk in chunks])
    return {"choices": [{"message": {"content": all_text, "role": "assistant"}}]}

@traceable(
    run_type="llm",
    metadata={"ls_provider": "my_provider", "ls_model_name": "my_model"},
    # TODO: Add a reduce_fn
)
def my_streaming_chat_model(messages: list):
    for chunk in ["Hello, " + messages[1]["content"]]:
        yield {
            "choices": [
                {
                    "message": {
                        "content": chunk,
                        "role": "assistant",
                    }
                }
            ]
        }

list(
    my_streaming_chat_model(
        [
            {"role": "system", "content": "You are a helpful assistant. Please greet the user."},
            {"role": "user", "content": "polly the parrot"},
        ],
    )
)

[{'choices': [{'message': {'content': 'Hello, polly the parrot',
     'role': 'assistant'}}]}]

### Retriever Runs + Documents

Many LLM applications require looking up documents from vector databases, knowledge graphs, or other types of indexes. Retriever traces are a way to log the documents that are retrieved by the retriever. LangSmith provides special rendering for retrieval steps in traces to make it easier to understand and diagnose retrieval issues. In order for retrieval steps to be rendered correctly, a few small steps need to be taken.

1. Annotate the retriever step with run_type="retriever".
2. Return a list of Python dictionaries or TypeScript objects from the retriever step. Each dictionary should contain the following keys:
    - page_content: The text of the document.
    - type: This should always be "Document".
    - metadata: A python dictionary or TypeScript object containing metadata about the document. This metadata will be displayed in the trace.

In [4]:
from langsmith import traceable

def _convert_docs(results):
  return [
      {
          "page_content": r,
          "type": "Document", # This is the wrong format! The key should be type
          "metadata": {"foo": "bar"}
      }
      for r in results
  ]

@traceable(
    # TODO: Add an run_type="retriever"
)
def retrieve_docs(query):
  # Retriever returning hardcoded dummy documents.
  # In production, this could be a real vector datatabase or other document index.
  contents = ["Document contents 1", "Document contents 2", "Document contents 3"]
  return _convert_docs(contents)

retrieve_docs("User query")

[{'page_content': 'Document contents 1',
  'type': 'Document',
  'metadata': {'foo': 'bar'}},
 {'page_content': 'Document contents 2',
  'type': 'Document',
  'metadata': {'foo': 'bar'}},
 {'page_content': 'Document contents 3',
  'type': 'Document',
  'metadata': {'foo': 'bar'}}]

### Tool Calling

LangSmith has custom rendering for Tool Calls made by the model to make it clear when provided tools are being used.

In [5]:
from langsmith import traceable
from openai import OpenAI
from typing import List, Optional
import json

openai_client = OpenAI()

@traceable(
  # TODO: Add an run_type="tool"
)
def get_current_temperature(location: str, unit: str):
    return 65 if unit == "Fahrenheit" else 17

@traceable(run_type="llm")
def call_openai(
    messages: List[dict], tools: Optional[List[dict]]
) -> str:
  return openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,
    tools=tools
  )

@traceable(run_type="chain")
def ask_about_the_weather(inputs, tools):
  response = call_openai(inputs, tools)
  tool_call_args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
  location = tool_call_args["location"]
  unit = tool_call_args["unit"]
  tool_response_message = {
    "role": "tool",
    "content": json.dumps({
        "location": location,
        "unit": unit,
        "temperature": get_current_temperature(location, unit),
    }),
    "tool_call_id": response.choices[0].message.tool_calls[0].id
  }
  inputs.append(response.choices[0].message)
  inputs.append(tool_response_message)
  output = call_openai(inputs, None)
  return output

tools = [
    {
      "type": "function",
      "function": {
        "name": "get_current_temperature",
        "description": "Get the current temperature for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": ["Celsius", "Fahrenheit"],
              "description": "The temperature unit to use. Infer this from the user's location."
            }
          },
          "required": ["location", "unit"]
        }
      }
    }
]
inputs = [
  {"role": "system", "content": "You are a helpful assistant."},
  {"role": "user", "content": "What is the weather today in New York City?"},
]

ask_about_the_weather(inputs, tools)

ChatCompletion(id='chatcmpl-CMy9KY1ShRmLRdoaPbcpRfd8SpuWQ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current temperature in New York City is 65°F. If you need more detailed weather information, feel free to ask!', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1759590466, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=25, prompt_tokens=83, total_tokens=108, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [10]:
from langsmith import traceable

@traceable(
    run_type="llm",
    metadata={"ls_provider": "comedy-central", "ls_model_name": "gpt-roast-master"}
)
def comedian_chat_bot(messages: list):
    # Simulate a comedian-themed LLM response
    user_message = messages[-1]["content"]
    
    if "politics" in user_message.lower() or "trump" in user_message.lower():
        return {
            "choices": [{
                "message": {
                    "role": "assistant", 
                    "content": "Why did Trump start a social media company? So he could finally have a platform where everyone agrees with him! The only thing bigger than his ego is his spray tan budget!"
                }
            }]
        }
    elif "election" in user_message.lower():
        return {
            "choices": [{
                "message": {
                    "role": "assistant", 
                    "content": "Trump's campaign strategy is like his hair - it defies gravity and makes you wonder how it's still standing! I heard he wants to build a wall around the debates so no tough questions get through."
                }
            }]
        }
    else:
        return {
            "choices": [{
                "message": {
                    "role": "assistant", 
                    "content": "I'm like Trump's tax returns - I've got all the best jokes but you never know what you're gonna get! What can I roast for you today?"
                }
            }]
        }

# Usage
messages = [
    {"role": "system", "content": "You are a stand-up comedian who specializes in political humor about Donald Trump"},
    {"role": "user", "content": "Tell me something funny about politics"}
]

response = comedian_chat_bot(messages)
print(response["choices"][0]["message"]["content"])

Why did Trump start a social media company? So he could finally have a platform where everyone agrees with him! The only thing bigger than his ego is his spray tan budget!


In [11]:
from langsmith import traceable

@traceable(
    run_type="retriever",
    metadata={"database": "trump-jokes", "category": "political-humor"}
)
def retrieve_trump_jokes(query: str):
    # Simulate retrieving political jokes from a database
    trump_jokes = [
        "Trump's memoir is called 'The Art of the Steal' - it's about both his business deals and his golf game!",
        "Why does Trump love Twitter? It's the only place where 280 characters can cause an international incident!",
        "Trump's idea of climate change is moving from Mar-a-Lago to Bedminster - that's a big temperature change!",
        "They asked Trump about AI regulation and he said 'I'm the smartest AI - Artificial Intelligence? More like Amazing Me!'",
        "Trump's border wall proposal now includes a moat filled with his legal documents - it's the only thing deeper than his denial!"
    ]
    
    # Simple keyword matching (in real app, this would be vector search)
    relevant_jokes = []
    for joke in trump_jokes:
        if any(word in query.lower() for word in ["trump", "politics", "president", "election"]):
            relevant_jokes.append(joke)
    
    # Return in proper document format for LangSmith
    return [
        {
            "page_content": joke,
            "type": "Document",
            "metadata": {"joke_id": i, "category": "political", "rating": "PG-13"}
        }
        for i, joke in enumerate(relevant_jokes[:3])  # Return top 3 jokes
    ]

# Usage
jokes = retrieve_trump_jokes("Tell me some Trump jokes")
for joke in jokes:
    print(f"Joke: {joke['page_content']}")
    print(f"Metadata: {joke['metadata']}\n")

Joke: Trump's memoir is called 'The Art of the Steal' - it's about both his business deals and his golf game!
Metadata: {'joke_id': 0, 'category': 'political', 'rating': 'PG-13'}

Joke: Why does Trump love Twitter? It's the only place where 280 characters can cause an international incident!
Metadata: {'joke_id': 1, 'category': 'political', 'rating': 'PG-13'}

Joke: Trump's idea of climate change is moving from Mar-a-Lago to Bedminster - that's a big temperature change!
Metadata: {'joke_id': 2, 'category': 'political', 'rating': 'PG-13'}

